## eDSL C: Layouting financial reports
#### Lef Ioannidis (@elefthei) : Investment Engineer, Bridgewater Associates

### eDSLs A, B introduced more types to maintain the logic semantics.

In [1]:
%classpath add mvn org.typelevel cats-core_2.12 2.1.1
%classpath add mvn joda-time joda-time 2.10.5
import org.joda.time.DateTime
import cats.Monoid
import cats.implicits._

import org.joda.time.DateTime
import cats.Monoid
import cats.implicits._


## But sometimes we want to **let go** of semantics in favor of better syntax.

## Case study: Formatting financial reports
##### "Everything is a widget!" 

In [ ]:
sealed trait Widget
case class Section(n: Int, s: Seq[Widget]) extends Widget
case class Chart(chart: Chart) extends Widget
case class Table(table: Table) extends Widget
case class Plot(plot: Plot) extends Widget
case class Markdown(s: String) extends Widget

### Let go of parametricity to get nicer syntax

In [ ]:
implicit class ComposeWidgets(first: Widget) {
    def <|>(second: Widget): Widget = Section(2, Seq(first, second))
    def <+>(second: Widget): Widget = Section(1, Seq(first, second))
    def <|>(second: String): Widget = first <|> Markdown(second)
    def <+>(second: String): Widget = first <+> Markdown(second)
}

implicit class StringWidgets(s: String) {
    def <|>(second: Widget): Widget = Markdown(s) <|> second
    def <+>(second: Widget): Widget = Markdown(s) <+> second
}
implicit class ComposeSeqWidgets(s: Iterable[Widget]){
    def horizontally: Widget = Section(s.size, s.toSeq)
    def vertically: Widget = Section(1, s.toSeq)
}

## Here's how it will look like

![title](img/widgets.png)

### All type information is lost (and that's ok)

### Let's also add currying for Maps, it will be helpful syntax.

In [ ]:
// Dummy implicits help overcome type erasure, nice hack but would be even better if Scala did not have this problem
final class DummyImplicit1 ()
implicit val dummy1 = new DummyImplicit1
final class DummyImplicit2 ()
implicit val dummy2 = new DummyImplicit2

// No erasure now
implicit class Map2Curry[A,B,V](m: Map[(A,B), V]) {
    def apply(a: A)(implicit dummy1: DummyImplicit1): Map[B, V] =
      m.groupBy(_._1._1)(a).map { case (k,v) => (k._2, v) }
    def apply(b: B)(implicit dummy2: DummyImplicit2): Map[A, V] =
      m.groupBy(_._1._2)(b).map { case (k,v) => (k._1, v) }
}

### Bring back PeriodSeries from eDSL A

In [ ]:
  sealed trait Frequency
  sealed abstract class FrequencyValue[F <: Frequency] {
    def days: Int
  }

  object Frequency {
    sealed trait Daily extends Frequency
    sealed trait Monthly extends Frequency
    sealed trait Quarterly extends Frequency
    sealed trait Yearly extends Frequency

    case object Daily extends FrequencyValue[Daily] {
      override def days: Int = 1
    }
    case object Monthly extends FrequencyValue[Monthly] {
      override def days: Int = 30
    }
    case object Quarterly extends FrequencyValue[Quarterly] {
      override def days: Int = 30 * 4
    }
    case object Yearly extends FrequencyValue[Yearly] {
      override def days: Int = 365
    }
  }
case class PeriodSeries[F <: Frequency](data: Array[Double], frequency: FrequencyValue[F], base: DateTime)

## Then I wrote some extensions for Maps, so we can connect this to eDSL B

In [ ]:
implicit class Map1Plot[A, F <: Frequency](m: Map[A, PeriodSeries[F]]) {
    def toChart: Widget = ??? // Some bridgwater UI specific code that's not very interesting
    def toPlotChart: Widget = ???
}

implicit class Map2Plot[A,B, F <: Frequency](m: Map[(A,B), PeriodSeries[F]]) {
    def toCharts: Widget = {
        def charts = for {
           key <- m.keys
        } yield m(key._1).toChart
        charts.horizontally
    }
    def toTable: Widget = ???
}

### Finished eDSL use looks like this

In [ ]:
type Company = String
type Code = DateTime

def CompanyReport(company: Company): Widget = {
    def companyPrices[F <: Frequency]: Map[(Company, Code), PeriodSeries[F]] = ???
    
    val dotchart: Widget = companyPrices(company).toPlotChart
    val table: Widget = companyPrices.toTable
    val charts: Widget = companyPrices.toCharts
    
    // Layouting with nice operators and no types
    s"$company three most important economic events" <+> 
      (table <|> dotchart) <+> 
      charts
}

### Questions?

![title](img/report.jpg)